# Check CUDA

In [2]:
import torch
import pandas as pd
import ast

print("CUDA: ", torch.cuda.is_available())

ModuleNotFoundError: No module named 'pandas'

# Load data and defined label

In [19]:
ner_tags = [
"MAT",
"NMAT",
"DIMENSION",
"WEIGHT",
"TARGET_USER",
"PROPERTY",
"COLOR",
"SHAPE",
"SIZE",
]

ner_tags_property = [
# "PROPERTY"
]

processed_ner_tags = ['O']
for tag in ner_tags:
        processed_ner_tags.extend([f"B-{tag}", f"I-{tag}"])

print(processed_ner_tags)
print(len(processed_ner_tags))

ner_tags_2_number = {t: i for (i, t) in enumerate(processed_ner_tags)}
number_2_ner_tags = {t: i for (t, i) in enumerate(ner_tags_2_number)}

['O', 'B-MAT', 'I-MAT', 'B-NMAT', 'I-NMAT', 'B-DIMENSION', 'I-DIMENSION', 'B-WEIGHT', 'I-WEIGHT', 'B-TARGET_USER', 'I-TARGET_USER', 'B-PROPERTY', 'I-PROPERTY', 'B-COLOR', 'I-COLOR', 'B-SHAPE', 'I-SHAPE', 'B-SIZE', 'I-SIZE']
19


In [20]:
raw_dataset = pd.read_excel("../data/raw_data_restore_uppercase.xlsx")
raw_dataset.head()

,sentence,locs,words
0,🐶 【 Support and Piping Design 】 : Supportive r...,"[('45', '57', 'PROPERTY'), ('114', '118', 'TAR...","['raised sides', 'cats', 'dogs', 'pet', 'recta..."
1,🐶 【 High - Quality Materials 】 : The inner rin...,"[('101', '105', 'MAT'), ('37', '47', 'PROPERTY')]","['wool', 'inner ring']"
2,🐶 【 Non - Slip Bottom 】 : The rectangular bed ...,"[('4', '21', 'PROPERTY'), ('59', '76', 'PROPER...","['Non - Slip Bottom', 'non - slip bottom', 're..."
3,🐶 【 Easy to Care 】 : The pet bed can be machin...,"[('40', '54', 'PROPERTY'), ('25', '28', 'TARGE...","['machine washed', 'pet']"
4,🐶 【 Pls Noted 】 : As the bed is vacuum - packe...,"[('177', '180', 'TARGET_USER')]",['pet']


In [21]:
from datasets import Dataset, DatasetDict, ClassLabel, Features, Value, Sequence

train_dataset = Dataset.from_pandas(raw_dataset)
val_dataset = Dataset.from_pandas(raw_dataset)
test_dataset = Dataset.from_pandas(raw_dataset)

raw_datasets = DatasetDict(
    {"train": train_dataset, "val": val_dataset, "test": test_dataset})
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'locs', 'words'],
        num_rows: 2784
    })
    val: Dataset({
        features: ['sentence', 'locs', 'words'],
        num_rows: 2784
    })
    test: Dataset({
        features: ['sentence', 'locs', 'words'],
        num_rows: 2784
    })
})

# Visualize data

In [22]:
def visualize_ner_tags(words, labels):
    line1 = ""
    line2 = ""
    for word, label in zip(words, labels):
        # print(word, label)
        label = str(label)
        max_length = max(len(word), len(label))
        line1 += word + " " * (max_length - len(word) + 1)
        line2 += label + " " * (max_length - len(label) + 1)
    print(line1)
    print(line2)

In [23]:
def assign_ner_tags_roberta(example):

    token_input = tokenizer(example['sentence'])
    example['tokens'] = tokenizer.convert_ids_to_tokens(
        token_input['input_ids'])

    ner_tags = [0 for token in example['tokens']]
    if str(type(example['locs'])) == "<class 'list'>":
        locs = example['locs']
    else:
        locs = ast.literal_eval(example['locs'])

    locs = [(int(loc[0]), int(loc[1]), loc[2]) for loc in locs]
    locs = sorted(locs)
    bg_id = 1
    pre_loc = 0
    text = example['sentence']
    for loc in locs:
        loc0 = int(loc[0])
        loc1 = int(loc[1])

        pre_text = text[pre_loc:loc0]
        if 0 < loc0 and len(pre_text) > 0 and pre_text[-1] == ' ':
            pre_text = text[pre_loc:loc0 - 1]
        token_input = tokenizer(pre_text)
        pre_token = tokenizer.convert_ids_to_tokens(
            token_input['input_ids'])
        bg_id = bg_id + len(pre_token) - 2
        pre_loc = loc1

        word = example['sentence'][loc0: loc1].strip()
        if loc0 > 0 and example['sentence'][loc0 - 1] == ' ':
            word = " " + word
        token_input = tokenizer(word)
        word_token = tokenizer.convert_ids_to_tokens(token_input['input_ids'])

        label_number = ner_tags_2_number[f"B-{loc[2]}"]
        ner_tags[bg_id] = label_number
        bg_id += 1
        for idx in range(bg_id, bg_id + len(word_token) - 3):
            ner_tags[idx] = label_number + 1
        bg_id = bg_id + len(word_token) - 3

        # visualize_ner_tags(example['tokens'], ner_tags)

    ner_tags[0] = -100
    ner_tags[-1] = -100
    return ner_tags

In [24]:
id = 1571
print(raw_datasets['train'][id]['sentence'])
print("-"*50)

token_input = tokenizer(raw_datasets['train'][id]['sentence'])
print(token_input)
print("-"*50)

words = tokenizer.convert_ids_to_tokens(token_input['input_ids'])
print(words)
labels = assign_ner_tags_roberta(raw_datasets['train'][id])

visualize_ner_tags(words, labels)

Safe Design : The four legs are welded with x - shaped rods on both top and base to ensure stability , and each end table leg has a non - slip rubber pad for floor protection and avoid noising your pets .
--------------------------------------------------
{'input_ids': [0, 32356, 7438, 4832, 20, 237, 5856, 32, 36736, 196, 19, 3023, 111, 14216, 33920, 15, 258, 299, 8, 1542, 7, 1306, 5443, 2156, 8, 349, 253, 2103, 2985, 34, 10, 786, 111, 9215, 11283, 11212, 13, 1929, 2591, 8, 1877, 117, 3009, 110, 10029, 479, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
--------------------------------------------------
['<s>', 'Safe', 'ĠDesign', 'Ġ:', 'ĠThe', 'Ġfour', 'Ġlegs', 'Ġare', 'Ġweld', 'ed', 'Ġwith', 'Ġx', 'Ġ-', 'Ġshaped', 'Ġrods', 'Ġon', 'Ġboth', 'Ġtop', 'Ġand', 'Ġbase', 'Ġto', 'Ġensure', 'Ġstability', 'Ġ,', 'Ġand', 'Ġeach', 'Ġend', 'Ġtable', 'Ġleg', 'Ġhas', 'Ġa', 'Ġnon', 'Ġ-'

# Preprocessing data

In [25]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["sentence"], truncation=True, is_split_into_words=False
    )
    tokenized_inputs["labels"] = assign_ner_tags_bert(examples)
    return tokenized_inputs

In [26]:
from datasets import Dataset

list_input_ids = []
list_token_type_ids = []
list_attention_mask = []
list_labels = []

for index, row in raw_dataset.iterrows():
    try:
        label = assign_ner_tags_roberta(row)
        token_input = tokenizer(row['sentence'])
        list_input_ids.append(token_input['input_ids'])
        # list_token_type_ids.append(token_input['token_type_ids'])
        list_attention_mask.append(token_input['attention_mask'])
        list_labels.append(label)
    except Exception as error:
        # print(error)
        print(index)
        print(row['sentence'])

tokenized_datasets = pd.DataFrame()
tokenized_datasets['input_ids'] = pd.Series(list_input_ids)
# tokenized_datasets['token_type_ids'] = pd.Series(list_token_type_ids)
tokenized_datasets['attention_mask'] = pd.Series(list_attention_mask)
tokenized_datasets['labels'] = pd.Series(list_labels)

train_dataset = Dataset.from_pandas(tokenized_datasets)
val_dataset = Dataset.from_pandas(tokenized_datasets)
test_dataset = Dataset.from_pandas(tokenized_datasets)

tokenized_datasets = DatasetDict(
    {"train": train_dataset, "val": val_dataset, "test": test_dataset})
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2784
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2784
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2784
    })
})

In [27]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [28]:
import evaluate

metric = evaluate.load("seqeval")

In [29]:
import numpy as np

label_names = processed_ner_tags
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100]
                   for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(
        predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [30]:
from transformers import AutoModelForTokenClassification
from transformers.models.bert import modeling_bert
from transformers import RobertaForTokenClassification

model = RobertaForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)
model.config.num_labels

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions

19

# Training loop

In [32]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], collate_fn=data_collator, batch_size=8
)

In [33]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [34]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [35]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [36]:
from transformers import get_scheduler

num_train_epochs = 30
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
from tqdm.auto import tqdm
import torch
from datetime import datetime
import os

date_time = datetime.now()
format_date = date_time.strftime('%Y-%m-%d')
format_time = date_time.strftime('%H:%M:%S')

print(f"Date: {format_date}")
print(f"Time: {format_time}")

# Replace with desire output dir
output_dir = f"../models/model_from_{format_date}/roberta-base_{format_time}"
progress_bar = tqdm(range(num_training_steps))

best_f1_score = 0

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(
            predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(
            labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(
            predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    if (epoch + 1) % 5 == 0:
        output_ckpt = os.path.join(output_dir, f'epoch_{epoch + 1}')
        print(f"Save model at epoch {epoch + 1}")
        tokenizer.save_pretrained(output_ckpt)
        unwrapped_model.save_pretrained(output_ckpt, save_function=accelerator.save)